In [138]:
import pandas as pd
f = open('dna.example.fasta')
fasta = f.read()
record = pd.DataFrame()
fasta_id = fasta.split('>') #splice into sequences
fasta_id
del fasta_id[0] #delate first item, which's ''
fasta_id[0].split('\n')[0].split(',')[0]
len(''.join(fasta_id[0].splitlines()[1:-1]))

980

In [139]:
#
Sequence_identifier = []
Sequence_description = []
Sequence = []
Seq_len = [] 
for sequence in fasta_id:
    lines = sequence.splitlines() #separate each line, first line(item) is the ID&description
    Sequence_identifier.append(lines[0].split(',')[0]) #separate by ',' first item is ID
    Sequence_description.append(lines[0].split(',')[1]) #second item is description
    Sequence.append( ''.join(lines[1:-1])) #join the rest items in the list as the sequence
    Seq_len.append(len(''.join(lines[1:-1])))
record['Seq_id'] = Sequence_identifier
record['Seq_description'] = Sequence_description
record['Sequence'] = Sequence
record['Sequence_length'] = Seq_len
record.head(8)

,Seq_id,Seq_description,Sequence,Sequence_length
0,gi|142022655|gb|EQ086233.1|43 marine metagenom...,whole genome shotgun sequence,TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGC...,980
1,gi|142022655|gb|EQ086233.1|160 marine metageno...,whole genome shotgun sequence,ATTGGGGAGGAGGCGAGTTGAGCGGCGGCAGTTCGCCTGCGTGCGC...,700
2,gi|142022655|gb|EQ086233.1|41 marine metagenom...,whole genome shotgun sequence,GACCTTGCATCGGCTGATCGCCGAGCGTGCCGACGTATTCATTCAC...,3010
3,gi|142022655|gb|EQ086233.1|221 marine metageno...,whole genome shotgun sequence,GCCCGGCGATTTGACGTCTGCAGCCTCACGTCCAAACTCAATTTGA...,2800
4,gi|142022655|gb|EQ086233.1|294 marine metageno...,whole genome shotgun sequence,GATCAGCCCCGCATACGCGTACGCGCGTGCGACGCCGAAGAGCGTC...,3780
5,gi|142022655|gb|EQ086233.1|323 marine metageno...,whole genome shotgun sequence,ACGCCCGGCGCACCGCGAGTACCGCGCCGCCGGGCACTCCTTGACC...,4760
6,gi|142022655|gb|EQ086233.1|564 marine metageno...,whole genome shotgun sequence,CTGGTAGCCATGCAGCAAGGCTGGCGCTAGATGTACGGCCAGATTG...,1610
7,gi|142022655|gb|EQ086233.1|521 marine metageno...,whole genome shotgun sequence,CGTTGTTCGCCAGGTCGTCCGCATAGCCGGCCGAGCTGAACTGCGT...,490


In [140]:
longest_seq = record[record['Sequence_length'] == max(record['Sequence_length'])]['Seq_id'].tolist()[0]#getting ID for the longest sequence
shortest_seq = record[record['Sequence_length'] == min(record['Sequence_length'])]['Seq_id'].tolist()[0]#getting ID for the shortest sequence
longest_seq

'gi|142022655|gb|EQ086233.1|323 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold'

In [251]:
'''This function takes in the sequence and the # of reading frame(1,2 or 3), first it searches for a start condon,
then, it searches for the nearest inframe stop codon, after finding the stop codon, it append the sequence between the 
start codon and stop codon to the ORF list'''
def checkORF(seq, frame): #giving the sequence and readingframe, return all ORF, # of ORF, length of the longest ORF
    ORF = []
    stop_codon = ['taa', 'tag','tga']
    for i in range(frame,len(seq)-3,3):
        if seq[i:i+3].lower() == 'atg':
            for j in range(i+3,len(seq)-3,3):
                if seq[j:j+3].lower() in stop_codon:
                    ORF.append(seq[i:j+3])
                    break
    return ORF, len(ORF), len(max(ORF, key = len))

def totalORF(seq): #total reading frame for all three possibility
    total_number = checkORF(seq,1)[1]+checkORF(seq,2)[1]+checkORF(seq,3)[1]
    all_ORF = checkORF(seq,1)[0]+checkORF(seq,2)[0]+checkORF(seq,3)[0]
    return all_ORF,total_number

def longestORF(seq): #return the longest ORF and the length
    return max(totalORF(seq)[0], key = len), len(max(totalORF(seq)[0], key = len))


    
checkORF(record['Sequence'][5],1)
#totalORF(record['Sequence'][5])
#longestORF(record['Sequence'][5])

(['ATGAAACCCGAAAACCTCGTCGCCTGCCACGAATGCGACCTGCTGTTTTGGCGGCCGCCGCGCTTGCGCGCGCTGGCTGCGCACTGCCCGAGGTGCCGTGCCCGCGTGGGCGGCAGCGCGCACGGCCGTCCGGCGCTCGACCGGCGGTGCGCGATCGCGCTCGCCGCGCTGTTCACGCTCTTCATCGCGCAGGCCTTTCCCATCGTCGCGCTCGACGCCGCCGGCATCGCATCGCACGCGACGCTGGCCGACGCGGTGGCCGCGTTGCGCTTGAACGGGCAACCGGCGGTGGCGGCGATCGTGTTCTGCACGACGATGTTGTTCCCGCTGCTGGAACTCGCCGCGTGGCTGTACGTGCTCGTACCGTTGCGCGCGGGCCGCGTACCGCCCCGCTTCGAGCCGGTCCTGCGCAACATGCAGCGGCTGCGCCCGTGGAGCATGGTCGAGGTGTTCCTGCTCGGCATCCTGGTCACGATCGTCAAGATGACGAGCCTCGCGCACGTGATACCGGGCCCCGCGCTGTTTGCGTTCGGCGCCCTCACCGTGTTGCTCGGCTTTCTCGCGTCATTCGACCCGGGCGGCCTGTGGGAAGCGCGCGACGAAATCATCGCGCTGCGCGGCGGCGGTACGTCCGCCGCGGTATCGCGCCGGCGGCACACGCCGCGACGCGCTGCACCGGTGACGCCCGACACAGCGGACGCAACGAACGCGACCGGCGCGACCGGGCGCCACCGCTCGGCAAGCGTCACGGCCCGCGCCGCGGGGCTGGTCGCATGTCATACCTGCGGACGCGTGCAACCGCATACCGAAGCCGCCGACGCGCGCTGCACGCGCTGCGGTTCGACGCTGCACGAACGCCGCCCGCGCAGCGCCGCACGCACGGGCGCGCTCGTCATCGCGGCCGCGCTTCTCTACATCCCGGCGAACCTGCTGCCCATCATGCATGCGACCTCGCTCGGCCGCGCCGAGGACGACACCATTCTCGCCGGCGTCGCGT

In [204]:
'''incooperate the ORF info into the dataframe
   - total reading frame
   - longest reading frame
   - start position of longest reading frame
   
   what is the length of the longest ORF in the file?
   What is the identifier of the sequence containing the longest ORF? 
   For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? 
   What is the starting position of the longest ORF in the sequence that contains it? '''

'incooperate the ORF info into the dataframe\n   - total reading frame\n   - longest reading frame\n   - start position of longest reading frame\n   \n   what is the length of the longest ORF in the file?\n   What is the identifier of the sequence containing the longest ORF? \n   For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? \n   What is the starting position of the longest ORF in the sequence that contains it? '